<a href="https://colab.research.google.com/github/jdweak/CSE198-CTF/blob/main/Final_CSE198CTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TODO: Password Cracking Algorithms (unhashed, brute force, dictionary, rainbow table) || Encryption: Sub cipher, ceasar shift, etc

# CTF: Password Cracking

To really understand password security and best practices, in this mock CTF we'll explore and implement password cracking techniques. Our dataset will draw from the OWASP project's dataset of the most common passwords (https://github.com/danielmiessler/SecLists/tree/master).

## Flag 1: Finding passwords in a plaintext list (1 pt)

A common security flaw in early websites was storing password databases in plaintext (without hashing). This meant that for any of these websites, if a hacker got access to the database (for example through SQL injection) they could easily find the password of any user regardless of password strength.

For this flag, look in the plaintext_passwords.csv file and find the password of user 36. When you find it assign the variable FLAG1 in the cell below to the password and run the cell.

In [ ]:
FLAG1 = "user 36's password"

## Flag 2: Basic Password Hashing & Dictionary Attack (3 points)

Flag 1 should have demonstrated why hashing passwords is so important to ensure security. With hashed passwords, instead of seeing a table full of everyone's password, you now see a bunch of long strings which were encrypted with an irreversable hashing algorithm (in this case sha256). So instead of seeing the plaintext of user 36's password in our table, we now see this:



```
User ID: 36 || SHA256 Hash: ed45d626b07112a8a501d9672f3b92796a6754b8d8d9cb4c617fec9774889220
```

Not so easy to get the credentials now. For this next challenge, we're going to implement a dictionary attack to crack a user's password. In this case, our dictionary will be the list of the top 1000 most common passwords. Use the provided hash_SHA256 method and list of passwords to find the plaintext of the hashed password below:


```
8bd10698229d26627eb039ac20f4537b62c6687ccb2892590bc7a3691659e892
```

Once you figure out the plaintext password, paste it into the FLAG2 variable below like you did in flag 1.


In [6]:
# Run this code to import password files needed for later
!git clone https://github.com/jdweak/CSE198-CTF.git

Cloning into 'CSE198-CTF'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 9.43 KiB | 9.43 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import hashlib

# This method allows you to hash any string using the SHA256 algorithm.
def hash_SHA256(password):
  sha256_hash = hashlib.sha256(password.encode('utf-8')).hexdigest()
  return sha256_hash

In [9]:
# Read the password file into a list called plaintexts
with open('/content/CSE198-CTF/top_1000_plaintext.csv', 'r') as f:
    plaintexts = [line.strip() for line in f]

# print out the first plaintext in the list
print(plaintexts[0])


# HINT: If you as a person were given a list of the most common passwords and instructions on how to hash a password,
# how would you do this task by hand?)

# Your code goes here


Plaintext


In [ ]:
FLAG2 = "plaintext password"

## Flag 3-5: Brute Force Attacks (8 points)
Dictionary attacks and rainbow tables are great ways to quickly crack passwords, but what if the hashed password you are trying to crack isn't in any of those lists? We fallback to the attack of last resort: the Brute Force Attack.

In this attack, we will simply try every possible combination of letters and numbers until we find a matching hash value. Theoretically, this attack will always succeed at cracking a password (since all possible combinations are attempted). In practice, strong passwords take so long to crack it's infeasible to actually decipher them using this strategy.

For flags 3-5, use a brute force cracking algorithm to find the plaintext of the following hashes. The length of the plaintext password for some flags will be given to make the coding a bit easier.


Flag 3 (1 point, password length 1 character):


```
6da43b944e494e885e69af021f93c6d9331c78aa228084711429160a5bbd15b5
```



Flag 4 (3 points, password length 3 characters):

```
6e0290d62f6db1779d6318df50209de8c9b93adb29b7dd46e7b563f044103b40
```


Flag 5 (4 points password length between 1 and 10 characters):


```
0283e655bc721952a780ab2a11a2c674c248d68918e7c2c070ac4cdd9b08a172
```



In [11]:
# All passwords will contain letters only from the below alphabet. A specific letter can be accessed by doing alphabet[idx]
alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
alphabet[0]

# HINT 1: How would you create a list of all 2 letter combinations after being given a list of all 1 letter combinations?
# How about a list of all 3 letter combinations given the 2 combination list? Try to generalize this concept of building
# combination lists off of previous lists in your code. If you are having trouble thinking about this in code try doing
# it by hand.

# HINT 2: For actually coding this concept, embedded loops or recursion are probably your friends

# Your code here


'a'

In [ ]:
FLAG3 = ""
FLAG4 = ""
FLAG5 = ""